In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os, random, shutil, sys, time
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import sys

#mac
util_path = "/Users/kimd999/research/projects/Katrina/srpAnalytics/code/latest/util"

#constance
'''args = sys.argv[0:]
py_file = args[0]

code_location = os.path.dirname(os.path.abspath(py_file))
index_of_latest = code_location.index('latest')
util_path = os.path.join(code_location[:index_of_latest], "latest", "util")
'''

sys.path.insert(0, util_path)

In [2]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/projects/Katrina/srpAnalytics/code/latest/2_bmd/2_analysis/Bree/CTG


In [4]:
#complete_file_path = '/Users/kimd999/research/projects/Katrina/per_each_data/Bree/run_2021_Nov/run_Nov_data_in_Nov/w_average_of_already_averaged_values/input/Nov_w_average_of_already_averaged_values.csv'

complete_file_path = '/Users/kimd999/research/projects/Katrina/per_each_data/Bree/run_2021_Nov/run_Nov_data_in_Nov/correct/input/Nov_w_average_values.csv'

df_test_dose_response = pd.read_csv(complete_file_path, header = 0)
pd.set_option('display.max_columns', None)
display(df_test_dose_response.head())
display(df_test_dose_response.tail())
#display(np.unique(morpho_data.well))

,dose,num_affected,total_num
0,0.0,10.536908,100
1,11.5,0.000000,100
2,23.0,8.223597,100
3,57.6,15.938307,100
4,92.2,62.630170,100


,dose,num_affected,total_num
3,57.6,15.938307,100
4,92.2,62.630170,100
5,115.2,62.030508,100
6,172.8,83.389839,100
7,230.4,97.021805,100


In [5]:
import generate_dose_response as gdr
import BMD_BMDL_estimation as bmdest
import Plot_Save as ps

In [6]:
# Fit dose response models
start_time = time.time()

df_test_dose_response = df_test_dose_response.sort_values(by=['num_affected'])
df_test_dose_response = df_test_dose_response.sort_values(by=['dose'])
print (f"df_test_dose_response:\n{df_test_dose_response}")

model_predictions = bmdest.analyze_dose_response_data(df_test_dose_response)
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("BMD calculation is done, it took:"+str(time_took)) 

df_test_dose_response:
    dose  num_affected  total_num
0    0.0     10.536908        100
1   11.5      0.000000        100
2   23.0      8.223597        100
3   57.6     15.938307        100
4   92.2     62.630170        100
5  115.2     62.030508        100
6  172.8     83.389839        100
7  230.4     97.021805        100
Model Convergence:True
BMD calculation is done, it took:3.3 seconds


In [7]:
print (model_predictions)

            Model  Chi-squared         p-val         AIC      BMD10  \
0        logistic    32.556469  1.276067e-05  636.573575  38.468094   
1           gamma    24.103197  2.074197e-04  628.434293  43.464468   
2         weibull          NaN           NaN         NaN        NaN   
3    log_logistic    22.723699  3.811883e-04  627.287499  47.850256   
4          probit    34.380343  5.680121e-06  638.255895  35.670867   
5      log_probit          NaN           NaN         NaN        NaN   
6    multistage_2    33.558499  2.914771e-06  641.190673  27.776240   
7  quantal_linear    75.911325  2.491510e-14  691.588859  12.898033   

      BMDL10       BMD50                                   Scaled Residuals  \
0        NaN   82.489446  [-2.1806261088715906, 2.8560859600367543, 0.64...   
1  35.955546   97.722743  [-2.2751092708714107, 2.435066715987379, -0.44...   
2        NaN         NaN           [nan, nan, nan, nan, nan, nan, nan, nan]   
3  40.668556   94.916424  [-2.13839204746861

In [8]:
# Select best model
selected_model_params = bmdest.select_model(model_predictions)
print (selected_model_params)

{'model': 3    log_logistic
Name: Model, dtype: object, 'no_unique_model_found_flag': 0, 'model_select_flag': 2, 'pval_thresh_flag': 1}


In [9]:
chemical_id = "Nov_w_raw_average_values"
end_point = "BMC"
bmd_feasibility_flag = 2

# Check if unique model is found
unique_model_flag = selected_model_params['no_unique_model_found_flag']

if(unique_model_flag == 0):
    # Generate report
    #print(df_test_dose_response.dose[-1:])

    ps.save_results_good_data_unique_model(df_test_dose_response, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
else:
    bmd_analysis_flag = selected_model_params['model_select_flag']
    if(bmd_analysis_flag == 1):
        ps.save_results_poor_data_or_no_convergence(df_test_dose_response, bmd_feasibility_flag, str(chemical_id), end_point, selected_model_params)
    else:
        ps.save_results_good_data_nounique_model(df_test_dose_response, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)